In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)
from yahooquery import Ticker

In [2]:
year = 2024
df = pd.read_csv(f'Data/sp500_wrds_{year}_fundamental.csv')
price_df = pd.read_csv(f'Data/sp500_wrds_{year}_price.csv') # include 12 months before data
# df.head()

In [3]:
len(df.columns)

76

In [4]:
# initial_columns = ['permno', 'adate', 'qdate', 'public_date', 'bm', 'evm', 'pe_op_basic',
#        'pe_op_dil', 'pe_exi', 'ps', 'pcf', 'dpr', 'npm', 'opmad', 'gpm',
#        'ptpm', 'cfm', 'roa', 'roe', 'roce', 'efftax', 'aftret_eq',
#        'aftret_invcapx', 'aftret_equity', 'pretret_noa', 'GProf',
#        'equity_invcap', 'debt_invcap', 'totdebt_invcap', 'capital_ratio',
#        'int_debt', 'int_totdebt', 'cash_lt', 'invt_act', 'rect_act', 'debt_at',
#        'debt_ebitda', 'short_debt', 'curr_debt', 'lt_debt', 'profit_lct',
#        'ocf_lct', 'cash_debt', 'fcf_ocf', 'lt_ppent', 'dltt_be', 'debt_assets',
#        'debt_capital', 'de_ratio', 'intcov', 'intcov_ratio', 'inv_turn',
#        'at_turn', 'rect_turn', 'pay_turn', 'sale_equity', 'sale_nwc',
#        'rd_sale', 'adv_sale', 'staff_sale', 'accrual', 'ptb', 'PEG_trailing',
#        'divyield', 'TICKER']
# # some are dropped due to high missing values or redundancy after this block

# df = df[initial_columns]

In [5]:
cols_to_drop = ['permno','adate', 'qdate','PEG_trailing','inv_turn', 'sale_nwc','profit_lct','ocf_lct','curr_debt', 'pretret_noa', 'intcov', 'intcov_ratio', 'rect_act','invt_act']

In [6]:
df = df.drop(columns=cols_to_drop)
df = df.rename(columns={'TICKER': 'ticker', 'public_date': 'date'})
df = df.dropna(subset=['ticker'])

In [7]:
# df.isna().sum()
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
# df.set_index('date', inplace=True)

In [8]:
#remoce % sign from divyield and convert to float
df["divyield"] = df["divyield"].str.rstrip('%')
df["divyield"] = df["divyield"].fillna(0)
df["divyield"] = df["divyield"].astype('float')

In [9]:
df = df.sort_values(['ticker', 'date'])
df = df.groupby('ticker').apply(lambda x: x.ffill().bfill())
df = df.reset_index(drop=True)

In [10]:
df.head()

,gvkey,date,CAPEI,bm,evm,pe_op_basic,pe_op_dil,pe_exi,pe_inc,ps,...,sale_invcap,sale_equity,rd_sale,adv_sale,staff_sale,accrual,ptb,divyield,ticker,cusip
0,126554,2024-01-31,34.692,0.195,19.343,29.908,30.116,31.050,31.050,5.579,...,0.786,1.169,0.070,0.008,0.0,-0.050,6.480,0.726,A,00846U10
1,126554,2024-02-29,36.633,0.195,19.343,31.577,31.796,32.783,32.783,5.891,...,0.786,1.169,0.070,0.008,0.0,-0.050,6.842,0.687,A,00846U10
2,126554,2024-03-31,39.881,0.162,23.184,33.297,33.605,34.645,34.645,6.331,...,0.787,1.153,0.072,0.008,0.0,-0.067,6.891,0.649,A,00846U10
3,126554,2024-04-30,37.559,0.162,23.184,31.359,31.649,32.629,32.629,5.963,...,0.787,1.153,0.072,0.008,0.0,-0.067,6.490,0.689,A,00846U10
4,126554,2024-05-31,35.742,0.162,23.184,29.842,30.118,31.050,31.050,5.674,...,0.787,1.153,0.072,0.008,0.0,-0.067,6.176,0.724,A,00846U10


2022 specific

In [11]:
# renam fb ticker to meta (2022 change)
df['ticker'] = df['ticker'].replace({'FB': 'META'})

In [12]:
print(len(df["ticker"].unique()))

468


In [13]:
price_df.columns

Index(['tic', 'datadate', 'gvkey', 'ggroup', 'gind', 'gsector', 'cshtrm',
       'prccm'],
      dtype='object')

In [14]:
# create a dictionary to map ticker to gind
gind_mapping = price_df.set_index('tic')['gind'].to_dict()
gsec_mapping = price_df.set_index('tic')['gsector'].to_dict()
df['gind'] = df['ticker'].map(gind_mapping)
df['gsector'] = df['ticker'].map(gsec_mapping)


price_df = price_df[['tic', 'datadate', 'cshtrm', 'prccm']]
price_df = price_df.rename(columns={'tic': 'ticker', 'datadate': 'date', 'cshtrm': 'volume', 'prccm': 'price'}) # rename columns
price_df['date'] = pd.to_datetime(price_df['date'], format='%Y-%m-%d')
# price_df.set_index('date', inplace=True)

In [15]:
group = price_df.groupby('ticker')

# 1. Simple Moving Averages
price_df['SMA_3']  = group['price'].transform(lambda x: x.rolling(3).mean())
price_df['SMA_6']  = group['price'].transform(lambda x: x.rolling(6).mean())
# price_df['SMA_12'] = group['price'].transform(lambda x: x.rolling(12).mean())

# 2. Price Returns / Momentum
price_df['ret_1m']  = group['price'].transform(lambda x: x.pct_change())
price_df['ret_3m']  = group['price'].transform(lambda x: x.pct_change(3))
price_df['ret_6m']  = group['price'].transform(lambda x: x.pct_change(6))
# price_df['ret_12m'] = group['price'].transform(lambda x: x.pct_change(12))

# 3. Volatility (Rolling Std of returns)
price_df['vol_3m']  = group['ret_1m'].transform(lambda x: x.rolling(3).std())
price_df['vol_6m']  = group['ret_1m'].transform(lambda x: x.rolling(6).std())
# price_df['vol_12m'] = group['ret_1m'].transform(lambda x: x.rolling(12).std())

# 4. Exponential Moving Average
price_df['EMA_6']  = group['price'].transform(lambda x: x.ewm(span=6, adjust=False).mean())
price_df['EMA_12'] = group['price'].transform(lambda x: x.ewm(span=12, adjust=False).mean())

# 5. RSI (approx price-only RSI)
def rsi(series, period=6):
    delta = series.diff()
    gain = delta.clip(lower=0).rolling(period).mean()
    loss = -delta.clip(upper=0).rolling(period).mean()
    rs = gain / loss
    return 100 - (100/(1 + rs))

price_df['RSI'] = group['price'].transform(rsi)

# 6. MACD
exp1 = group['price'].transform(lambda x: x.ewm(span=12, adjust=False).mean())
exp2 = group['price'].transform(lambda x: x.ewm(span=26, adjust=False).mean())
price_df['MACD_ratio'] = (exp1 - exp2) / price_df['price']


# 7. Volume moving average
price_df['vol_SMA_3'] = group['volume'].transform(lambda x: x.rolling(3).mean())
price_df['vol_SMA_6'] = group['volume'].transform(lambda x: x.rolling(6).mean())

# 8. Volume Rate of Change
price_df['vol_ROC'] = group['volume'].transform(lambda x: x.pct_change())

# 9. OBV (On-Balance Volume)
def obv(df):
    sign = df['price'].diff().apply(lambda x: 1 if x>0 else -1 if x<0 else 0)
    return (sign * df['volume']).cumsum()

price_df['OBV'] = group.apply(obv).reset_index(level=0, drop=True)


In [16]:
price_df.columns

Index(['ticker', 'date', 'volume', 'price', 'SMA_3', 'SMA_6', 'ret_1m',
       'ret_3m', 'ret_6m', 'vol_3m', 'vol_6m', 'EMA_6', 'EMA_12', 'RSI',
       'MACD_ratio', 'vol_SMA_3', 'vol_SMA_6', 'vol_ROC', 'OBV'],
      dtype='object')

In [17]:
nans_col = df.isna().sum()
missing_columns = nans_col[nans_col > 0]

for col in missing_columns.index:
    if col not in ['gind', 'gsector']:
        # 1. Fill using industry mean
        df[col] = df[col].fillna(df.groupby('gind')[col].transform('mean'))
        # 2. Fill remaining using sector mean
        df[col] = df[col].fillna(df.groupby('gsector')[col].transform('mean'))
        

In [18]:
df.columns

Index(['gvkey', 'date', 'CAPEI', 'bm', 'evm', 'pe_op_basic', 'pe_op_dil',
       'pe_exi', 'pe_inc', 'ps', 'pcf', 'dpr', 'npm', 'opmbd', 'opmad', 'gpm',
       'ptpm', 'cfm', 'roa', 'roe', 'roce', 'efftax', 'aftret_eq',
       'aftret_invcapx', 'aftret_equity', 'pretret_earnat', 'GProf',
       'equity_invcap', 'debt_invcap', 'totdebt_invcap', 'capital_ratio',
       'int_debt', 'int_totdebt', 'cash_lt', 'debt_at', 'debt_ebitda',
       'short_debt', 'lt_debt', 'cash_debt', 'fcf_ocf', 'lt_ppent', 'dltt_be',
       'debt_assets', 'debt_capital', 'de_ratio', 'cash_ratio', 'quick_ratio',
       'curr_ratio', 'cash_conversion', 'at_turn', 'rect_turn', 'pay_turn',
       'sale_invcap', 'sale_equity', 'rd_sale', 'adv_sale', 'staff_sale',
       'accrual', 'ptb', 'divyield', 'ticker', 'cusip', 'gind', 'gsector'],
      dtype='object')

In [19]:
# df.isna().sum()

In [20]:
# all other missing values imputed using KNN imputer
from sklearn.impute import KNNImputer

num_cols = df.select_dtypes(include='number').columns
imputer = KNNImputer(n_neighbors=3)
df[num_cols] = imputer.fit_transform(df[num_cols])

# still check for any missing values then simple fillna with column mean for numerical columns
df.drop(['gind', 'gsector'], axis=1, inplace=True)
num_cols = df.select_dtypes(include='number').columns
for col in num_cols:
    df[col] = df[col].fillna(df[col].mean())

In [21]:
# df.isna().sum()

In [22]:
price_tickers = price_df['ticker'].unique()
fundamentals_tickers = df['ticker'].unique()
common_tickers = set(price_tickers).intersection(set(fundamentals_tickers))
print(f"Number of unique tickers in fundamentals data: {len(fundamentals_tickers)}")
print(f"Number of unique tickers in price data: {len(price_tickers)}")
print(f"Number of common tickers: {len(common_tickers)}")

Number of unique tickers in fundamentals data: 468
Number of unique tickers in price data: 517
Number of common tickers: 453


In [23]:
# Need to manually check which tickers are missing in either dataset
missing_tickers = set(fundamentals_tickers) - common_tickers
print(f"Tickers in fundamentals data but missing in price data: {missing_tickers}")

Tickers in fundamentals data but missing in price data: {'FI', 'ITI', 'LEE', 'WW', 'CAL', 'AMR', 'APLD', 'PARA', 'WABC', 'COOP', 'ATI', 'CMT', 'PAR', 'BH', 'MLSS'}


In [24]:
df = df.set_index(['date', 'ticker'])
price_df = price_df.set_index(['date', 'ticker'])
df_merged = df.join(price_df, how='inner')
df_merged.reset_index(inplace=True)

In [25]:
df_merged.shape, len(df_merged['ticker'].unique())

((5260, 79), 453)

In [26]:
# df_merged.isna().sum()

In [27]:
df_merged.isna().sum()

date                 0
ticker               0
gvkey                0
CAPEI                0
bm                   0
evm                  0
pe_op_basic          0
pe_op_dil            0
pe_exi               0
pe_inc               0
ps                   0
pcf                  0
dpr                  0
npm                  0
opmbd                0
opmad                0
gpm                  0
ptpm                 0
cfm                  0
roa                  0
roe                  0
roce                 0
efftax               0
aftret_eq            0
aftret_invcapx       0
aftret_equity        0
pretret_earnat       0
GProf                0
equity_invcap        0
debt_invcap          0
totdebt_invcap       0
capital_ratio        0
int_debt             0
int_totdebt          0
cash_lt              0
debt_at              0
debt_ebitda          0
short_debt           0
lt_debt              0
cash_debt            0
fcf_ocf              0
lt_ppent             0
dltt_be              0
debt_assets

In [28]:
# show tickers with a single missing values
missing_tickers = df_merged[df_merged.isna().any(axis=1)]['ticker'].unique()
missing_tickers # investigate these tickers

array(['AIG', 'APD', 'APH', 'AZO', 'BLK', 'CDNS', 'CE', 'CEG', 'CINF',
       'CMCSA', 'CMI', 'CSGP', 'CVX', 'CZR', 'DLTR', 'DVA', 'DVN', 'EMN',
       'EMR', 'FDS', 'GEV', 'HAL', 'HES', 'HII', 'HON', 'ILMN', 'IP',
       'IPG', 'JBHT', 'JBL', 'JNPR', 'KLAC', 'KMI', 'LII', 'LKQ', 'LNT',
       'MCO', 'META', 'MHK', 'MLM', 'MPC', 'MRO', 'NI', 'NKE', 'PEG',
       'PH', 'PPL', 'QRVO', 'RF', 'ROL', 'RSG', 'SNPS', 'SO', 'SOLV',
       'SPGI', 'STT', 'SWKS', 'SYF', 'TDG', 'TFC', 'TJX', 'TSLA', 'TYL',
       'UAL', 'UBER', 'UNP', 'URI', 'VLO', 'VLTO', 'VST', 'WM', 'WRB',
       'WYNN', 'XEL', 'YUM'], dtype=object)

In [29]:
df.isna().sum()

gvkey              0
CAPEI              0
bm                 0
evm                0
pe_op_basic        0
pe_op_dil          0
pe_exi             0
pe_inc             0
ps                 0
pcf                0
dpr                0
npm                0
opmbd              0
opmad              0
gpm                0
ptpm               0
cfm                0
roa                0
roe                0
roce               0
efftax             0
aftret_eq          0
aftret_invcapx     0
aftret_equity      0
pretret_earnat     0
GProf              0
equity_invcap      0
debt_invcap        0
totdebt_invcap     0
capital_ratio      0
int_debt           0
int_totdebt        0
cash_lt            0
debt_at            0
debt_ebitda        0
short_debt         0
lt_debt            0
cash_debt          0
fcf_ocf            0
lt_ppent           0
dltt_be            0
debt_assets        0
debt_capital       0
de_ratio           0
cash_ratio         0
quick_ratio        0
curr_ratio         0
cash_conversi

In [30]:
# droping these tickers
df_merged = df_merged[~df_merged['ticker'].isin(missing_tickers)]
df_merged.isna().sum().sum()

0

In [31]:
# if above is not 0, see why

In [32]:
# save csv to be used in modeling
df_merged.to_csv(f'Data/sp500_cleaned_{year}.csv', index=False)

In [33]:
df_merged.shape

(4402, 79)